<a href="https://colab.research.google.com/github/sglhsr/Machine_Learing/blob/main/20220323_colab03b%E4%BD%BF%E7%94%A8%E7%B4%85%E6%A8%93%E5%A4%A2%E7%94%9F%E6%88%90%E5%99%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. 讀入使用套件

第一輪是讀進我們基本套件, 第二輪是 TensorFlow 用到的套件。

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
import pickle

from tensorflow.keras.models import load_model, model_from_json

### 2. 讀入訓練好的 RNN 紅樓夢生成器

這裡是參考[《精通機器學習-使用 Scikit-Learn, Keras 與 TensorFlow》](https://www.books.com.tw/products/0010854043?sloc=main)這本書中莎士比亞生成器的部份寫成的。架構很簡單,就每輸入 100 個字, 預測下一個字是什麼。雙層 LSTM, 每層 128 個神經元。訓練 10 次, 在 1080Ti GPU 的電腦上大概花了 10 個小時。

以下會從我的 GitHub 重新讀入模型, 已下載模型請直接跳到下一段。


In [ ]:
from urllib.request import urlretrieve

In [ ]:
urlretrieve("https://raw.githubusercontent.com/yenlung/Deep-Learning-Basics/master/dream_rnn_architecture.json", "architecture.json")
urlretrieve("https://github.com/yenlung/Deep-Learning-Basics/raw/master/dream_rnn_weights.h5", "weights.h5")
urlretrieve("https://github.com/yenlung/Deep-Learning-Basics/raw/master/dream_tokenizer2.pkl", "tokenizer.pkl")

('tokenizer.pkl', <http.client.HTTPMessage at 0x7fc6292f0790>)

In [ ]:
f = open('architecture.json', 'r')
loaded_model = f.read()
f.close()

In [ ]:
model = model_from_json(loaded_model)

In [ ]:
model.load_weights("weights.h5")

In [ ]:
f = open('tokenizer.pkl', 'rb')
tokenizer = pickle.load(f)
f.close()

#### 【已下載紅樓夢生成器請執行這段】

如果你已經在[我的 GitHub](https://github.com/yenlung/Deep-Learning-Basics) 中下載紅樓夢生成模型, 這包括:

1. `dream_rnn` 資料夾, 這是模型和訓練好的權重。
2. `dream_tokenizer2.pkl` 檔案, 這是模型使用的 tokenizer。

存到你的 Google Drive, 放 Colab 程式的地方 (預設是 `Colab Notebooks` 資料夾)。那執行下面這一段, 不用再讀入一次模型資料。

In [ ]:
#from google.colab import drive

#drive.mount('/content/drive')

In [ ]:
#%cd '/content/drive/MyDrive/Colab Notebooks/'

In [ ]:
#model = load_model('dream_rnn')

In [ ]:
#f = open('dream_tokenizer2.pkl', 'rb')
#tokenizer = pickle.load(f)
#f.close()

### 4. 製造紅樓夢生成器

首先 `max_id` 是記錄《紅樓夢》用到的所有不同的中文字字數, 包括新式標點符號。很讓人驚訝 (?) 的是, 字數並沒有想像中多。

In [ ]:
max_id = len(tokenizer.word_index)

接下來是一段文字, 我們用事先訓練好的 tokenizer 換成一段數字, 最後用 one-hot encoding 回傳。

In [ ]:
#換成數字用Embadding,但這是換one hot encoding矩陣
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences([texts]))-1
    return tf.one_hot(X, max_id)

這段程式主要依輸入的一段文字, 用我們的 model 去預測下一個字。注意像平常的分類問題, 這裡輸出是每人個字出現機率最高的。但都照這樣, 我們輸入同一段文字, 之後出現的文字永遠是一樣的! 常用的手法是去設定 `temperature`, `temperature` 接近 0, 大致上就取機率最高的字; `temperature` 越大就越隨機。太隨機就變成亂數取字!就變成用抽籤的也不用訓練了 一般 `temperature`設 1 左右效果最佳。

In [ ]:
def next_char(texts, temperature=1):
    X_new = preprocess(texts)
    y_predict = model.predict(X_new)[0, -1:, :]
    #[1,3,4522]=>[1,4522]第一維(也只有一個)的最後一筆的全部 =  抓最後一批陣列
    #temperature稍微亂數 太小會固定那些字的組合,太大完全無規則
    rescaled_logits = tf.math.log(y_predict) / temperature
    #所有類的未正規化的log概率
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1  #隨機取一個字,預測每段字的下個字
    #tf.random.categorical：從一個分類分布中抽取樣本
    #num_samples從每一行切片中抽取的獨立樣本的数量
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

最後就一段文字進來，再產生 `n_chars` 這麼多個字。我們原本一段文字只能生一個字, 那就一次生一個字, 最後要生多少個字就生多少個字。

原本訓練我們一段是 100 個字去訓練的, 這裡超過 100 字時我們就取最後 100 個字丟入模型。

In [ ]:
def complete_text(texts, n_chars=50, temperature=1):
    n_chars=int(n_chars)
    for _ in range(n_chars):
        texts = texts + next_char(texts[-100:], temperature)
    return texts

做成 web app 前, 先來測試一下。

In [ ]:
complete_text("自孫悟空從石頭中蹦出來之後，", n_chars=300, temperature=0.2)


'自孫悟空從石頭中蹦出來之後，說道：「寶玉既有如此，但未知之言，也是一樁奇事。這裡離草庵不遠，何以如此？」士隱道：「此事不知。」雨村聽了，益發驚異：「請問仙長，何曾與他結局俱屬平常呢？」士隱道：「此事不知，何以如此？」士隱道：「此事不知。」雨村聽了，益發驚異：「請問仙長，何必如此？」士隱道：「此事不知。」雨村聽了，益發驚異：「請問仙長，何必問你？」士隱道：「此事傳出，但是敝族閨秀，如此之多，何元妃以下，凡一劍斬斷、平的、平處。那善劉貴鄉少不得人，固是有奇所，所以在這裡。」\n\n那日，賈政已知賈母的事，王夫人便將寶釵的話說了。\n\n賈環聽了，益發驚異：「請問大夫，說是甄老爺、寶玉的話，我們這樣人家，不知道的，我們這樣一個人，這樣'

### 5. 用 `gradio` 做成一個網路 app

我們準備用 [`gradio`](https://gradio.app/) 套件, 神速做完一個 web app。最酷的是, 最後出現 `https://xxxx.gradio.app` 那個網址, 在你的 Colab 還在執行的時候, 任何人都可以用任何瀏覽器連進來使用!

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

In [ ]:
iface = gr.Interface(
    fn=complete_text,
    inputs=[
        "text",
        gr.inputs.Slider(50, 200, 1, 50),  #50~200個字,間隔1個字,預設50個
        gr.inputs.Slider(0.2, 2, 0.2, 1)], #temperature設定 範圍0.2~2, 每次隔0.2 預設1
    outputs="text",
    title="紅樓夢生成器",
    description="起個頭, 幫你完成一段紅樓夢。可以改變 temperature, 越小生出的字越固定, 越大越隨機。")
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://15970.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7860/',
 'https://15970.gradio.app')